In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import faiss
import torch
import wget
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf
from pytorch_lightning import Trainer
from IPython.display import display
from tqdm import tqdm

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager


# In[2]:


# # Download data into project directory
PROJECT_DIR = "." #Change if you don't want the current directory to be the project dir
DATA_DIR = os.path.join(PROJECT_DIR, "DataLeo")


# In[5]:


# Download config
# wget.download("https://raw.githubusercontent.com/vadam5/NeMo/main/examples/nlp/entity_linking/conf/tiny_example_entity_linking_config.yaml",
#               os.path.join(PROJECT_DIR, "tiny_example_entity_linking_config.yaml"))

# Load in config file
cfg = OmegaConf.load(os.path.join(PROJECT_DIR, "tiny_example_entity_linking_config (1).yaml"))

# Set config file variables
cfg.project_dir = PROJECT_DIR
cfg.model.nemo_path = os.path.join(PROJECT_DIR, "sap_bert_model.nemo")
cfg.model.train_ds.data_file = os.path.join(DATA_DIR, "train.txt")
cfg.model.validation_ds.data_file = os.path.join(DATA_DIR, "val.txt")

# Initialize the trainer and model
trainer = Trainer(**cfg.trainer)
exp_manager(trainer, cfg.get("exp_manager", None))
model = nemo_nlp.models.EntityLinkingModel(cfg=cfg.model, trainer=trainer)


# In[6]:


# Train and save the model
trainer.fit(model)
model.save_to(cfg.model.nemo_path)


from huggingface_hub import HfApi
api = HfApi()
username = api.whoami()['name']

# In[ ]:
MODEL_NAME = "SelfAlignmentPretrainingForMedicalEntityLinking-ClaraMeD-FineTuned"

try:
  api.create_repo(repo_id=MODEL_NAME)
  print("Successfully created repository !")
except Exception as e:
  print("Repository is possibly already created. Refer to error here - \n\n", e)

from huggingface_hub import Repository
local_dir = f'model-{MODEL_NAME}/'
hf_model_name = f'{username}/{MODEL_NAME}'

commit_message = "Upload model"
model_filename = f'{MODEL_NAME}.nemo'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  model.save_to(model_filename)



[NeMo W 2024-04-23 12:45:05 nemo_logging:349] /home/joheras/.local/lib/python3.10/site-packages/hydra/core/plugins.py:225: UserWarning: 
    	Error importing 'hydra_plugins.hydra_colorlog'.
    	Plugin is incompatible with this Hydra version or buggy.
    	Recommended to uninstall or upgrade plugin.
    		ImportError : cannot import name 'SearchPathPlugin' from 'hydra.plugins' (/home/joheras/.local/lib/python3.10/site-packages/hydra/plugins/__init__.py)
      warnings.warn(
    



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


2024-04-23 12:45:07.087721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[NeMo W 2024-04-23 12:45:08 nemo_logging:349] /home/joheras/.conda/envs/hf/lib/python3.10/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
      rank_zero_warn(
    
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2024-04-23 12:45:08 exp_manager:394] Experiments will be logged at SelfAlignmentPretrainingTinyExample/2024-04-23_12-45-08
[NeMo I 2024-04-23 12:45:08 exp_manager:835] TensorboardLogger has been set up
[NeMo I 2024-04-23 12:45:09 entity_linking_dataset:78] Loaded dataset with 171466 examples
[NeMo I 2024-04-23 12:45:09 entity_linking_dataset:78] Loaded dataset with 1000 examples


[NeMo W 2024-04-23 12:45:09 nlp_overrides:457] Apex was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/apex
    Megatron-based models require Apex to function correctly.
[NeMo W 2024-04-23 12:45:09 lm_utils:91] bert-base-multilingual-uncased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


RuntimeError: Lightning can't create new processes if CUDA is already initialized. Did you manually call `torch.cuda.*` functions, have moved the model to the device, or allocated memory on the GPU any other way? Please remove any such calls, or change the selected strategy. You will have to restart the Python kernel.